In [46]:
import pandas as pd
import sys
import csv
import numpy as np
# 解除CSV字段限制
csv.field_size_limit(sys.maxsize)
# 定义常量
input_disc = "/home/luolintao/S20_NUMTs-detection-1.0/output/Y21100000492606.deduped.mt.disc.sam"



In [52]:
# 模块2：读取数据并初始过滤
df_raw = pd.read_csv(
    input_disc,
    sep="\t",
    names=['QNAME', 'FLAG', 'RNAME', 'POS', 'MAPQ', 'CIGAR', 'RNEXT', 'PNEXT', 'TLEN', 'SEQ', 'QUAL', 'SM', 'RG', 'NM', 'BC', 'OC', 'ZX', 'ZY', 'SA'],
    comment="@",
    low_memory=False
)
print("原始数据维度:", df_raw.shape)
df_raw.head(2)  # 显示前两行


原始数据维度: (2628302, 19)


,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL,SM,RG,NM,BC,OC,ZX,ZY,SA
0,DP8400025145TRL1C001R00300004294,97,10,42396563,38,150M,=,42385526,-10889,ATGCAATCGAATACAATCATCGAATTTACTCGTATGGAATCATCGA...,<).BC:DD3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DP8400025145TRL1C001R00300004294,145,10,42385526,0,150M,=,42396563,10889,AAGTAATCATCATGAAATGGAATCAAAAATAAACATCATCAATTGG...,FEEAEE8D:CDGE:FEECFDEEDEAEEEBDCAED=B;DCCFEADED...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# 初始过滤：仅保留有效染色体
valid_chromosomes = [str(i) for i in range(1, 23)] + ['X', 'Y', 'MT']
# 优化后
valid_chromosomes_set = set(valid_chromosomes)  # 集合查找O(1)
df_filtered_initial = df_raw[df_raw['RNAME'].astype(str).isin(valid_chromosomes_set)]

In [34]:
# 模块3：修复RNEXT字段
mask_eq = (df_filtered_initial['RNEXT'] == '=')
mask_valid_chr = df_filtered_initial['RNAME'].isin(valid_chromosomes[:-1])

df_fixed = df_filtered_initial.assign(
    RNEXT=np.where(
        mask_eq,
        np.where(mask_valid_chr, df_filtered_initial['RNAME'], 'MT'),
        df_filtered_initial['RNEXT']
    )
)

In [35]:
df_fixed

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL,SM,RG,NM,BC,OC,ZX,ZY,SA
0,DP8400025145TRL1C001R00300004294,97,10,42396563,38,150M,10,42385526,-10889,ATGCAATCGAATACAATCATCGAATTTACTCGTATGGAATCATCGA...,<).BC:DD3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DP8400025145TRL1C001R00300004294,145,10,42385526,0,150M,10,42396563,10889,AAGTAATCATCATGAAATGGAATCAAAAATAAACATCATCAATTGG...,FEEAEE8D:CDGE:FEECFDEEDEAEEEBDCAED=B;DCCFEADED...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DP8400025145TRL1C001R00300004532,97,10,42597038,17,150M,10,42599662,2774,CCCGAATGGAATCATCTAATGGAATGGAATGGAATAATCCATGGAC...,DGCEBGFFCCEGEFGFHFEFFDEFGFFDEEGE>FHGGGGGEFF;CE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DP8400025145TRL1C001R00300004532,145,10,42599662,22,150M,10,42597038,-2774,ATTGCATGGAATCATCATAAAATGGAATCGAATGGAATCAACATCA...,ECGFFIDFFFHEHHEHFFFHFFGEGEGEFGGFEFEGF<FDEFCDFD...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DP8400025145TRL1C001R00300005973,81,9,69710894,0,150M,8,43835483,0,TAGTATCTGGAAGTGGACATTTGGAGCGCTTTCAGGCCTATTTTGG...,CFEGF<=GDFAFDFFA>-B<FDEAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2628297,DP8400025145TRL1C057R05701976236,129,4,49155447,0,55S95M,2,89879893,0,ATTCCATTCCATTCCGTTCCTTTCCATTCCATTCCATTCCATTCTG...,B;C9EGD7DBFFAFEGE'8FDEDDF2FCHEE9F<E>:?GEEEEFBG1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2628298,DP8400025145TRL1C057R05701983114,65,4,49123102,6,24M3I2M2I79M40S,4,49642140,519039,GGGTTGACTCCATTCCATTCCATTCGAATCCATTCTGTTCCATTCC...,DEEFEFDFGFFGGFFFIEGEDIGHGG?FGGFFGFGHFFFHGFGHFG...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2628299,DP8400025145TRL1C057R05701983114,129,4,49642140,1,83M67S,4,49123102,-519039,AATGGAATGGAATGGTATGGAATAGAATGGAATGGAATGGAATACA...,EIIDAFIFGFFEHDHIHHGI8GIIFGGHFEFIHI?HHIHF;IGIIF...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2628300,DP8400025145TRL1C057R05702009334,81,4,49098842,0,93M10D46M11S,Y,58972967,0,TCCATTGCACACGGGTTCATTCCATTCCATTCCATTCCATTCCATT...,>DFCBCDD(,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# 模块4：最终过滤
mask = (
    (df_fixed['RNAME'].isin(valid_chromosomes[:-1])) & 
    (df_fixed['RNEXT'] == 'MT')
) | (
    (df_fixed['RNAME'] == 'MT') & 
    (df_fixed['RNEXT'].isin(valid_chromosomes[:-1]))
)

In [37]:
df_filtered = df_fixed[mask].copy()
print("最终过滤后数据量:", len(df_filtered))
df_filtered[['RNAME', 'RNEXT']].value_counts()
df_filtered

最终过滤后数据量: 614


,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL,SM,RG,NM,BC,OC,ZX,ZY,SA
14804,DP8400025145TRL1C001R02101822494,113,6,44154061,60,150M,MT,30,0,CCTTCCCCCCATTATCTCCACCCCACGCCGCCTACAGAGGAATCGT...,C=DB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14805,DP8400025145TRL1C001R02101822494,177,MT,30,60,150M,6,44154061,0,TCTCGGGAGCTCTCCATGCATTTGGTATTTTCGTCTGGGGGGTATG...,->'8CC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30986,DP8400025145TRL1C001R04500879707,65,MT,1028,0,150M,1,113288803,0,GTGGCTTTAACATATCTGAACACACAATAGCTATGACCCAAACTGG...,E8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30987,DP8400025145TRL1C001R04500879707,129,1,113288803,58,95M55S,MT,1028,0,CCGGCTGATTTTGTATTTTTAGTAGAGATGGGGTTTCTCCATGCTG...,CDDADCDBCDE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31894,DP8400025145TRL1C001R04601143936,81,4,176749568,60,150M,MT,3758,0,TTTTAATTGTTCCTGAAATAGAAAAAAACTAAGAAGATGACAATCT...,"DDDG;8AFE>8?DB<ECDEDF?D68D,DBDCD?EE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2620429,DP8400025145TRL1C057R04200307387,177,15,48730355,60,30S120M,MT,8331,0,GGGTTGCTATAGGGTAAATACGGGCCCTATTATTTCAAAGTAACCC...,"B/),2238G3=DDEFEFGH8DEFEBE?HAFHGGEFDFGIFIFFDAC...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2625454,DP8400025145TRL1C057R05000892984,97,MT,53,60,113M37S,3,36557688,0,GGTATTTTCGTCTGGGGGGTATGCACGCGATAGCATTGCGAGACGC...,C>EGEDEEFEDEEECDEBE9FEEGDEGDDBGDDEDEEEED<ADEEF...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2625455,DP8400025145TRL1C057R05000892984,145,3,36557688,60,8S142M,MT,53,0,TGGCCTCTCTGGTCTCAAACTCCTGACCTCGTGATCCTCCCGTCTC...,E&2BE&'1-C?6),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2625508,DP8400025145TRL1C057R05001066970,81,MT,13111,54,5S145M,15,100777510,0,TTTTTTTACTCATCCGCTTCCACCCCCTAGCAGAAAATAGCCCACT...,CAFCD:EBB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# 优化后（利用numpy向量化）
def cluster(data, maxgap=500):
    if not data:
        return []
    data = np.sort(data)
    breaks = np.where(np.diff(data) > maxgap)[0] + 1
    clusters = np.split(data, breaks)
    return [c.tolist() for c in clusters if len(c) >= 1]

In [39]:

# 按染色体分组并聚类
candidate_regions = []
for chr_name in df_filtered['RNAME'].unique():
    # 仅处理核染色体（排除MT）
    if chr_name == 'MT':
        continue
    
    # 提取当前染色体的读段
    chr_df = df_filtered[df_filtered['RNAME'] == chr_name]
    positions = chr_df['POS'].astype(int).tolist()
    
    # 聚类
    clusters = cluster(positions, maxgap=500)
    
    # 记录候选区域
    for cluster_points in clusters:
        if len(cluster_points) >= 2:  # 至少2个读段
            start = min(cluster_points) - 500  # 扩展500bp
            end = max(cluster_points) + 500
            candidate_regions.append({
                'chr': chr_name,
                'start': start,
                'end': end,
                'num_reads': len(cluster_points),
                'reads': cluster_points
            })

print("候选区域数量:", len(candidate_regions))


候选区域数量: 5


In [40]:
candidate_regions

[{'chr': '4',
  'start': 190597616,
  'end': 190598924,
  'num_reads': 7,
  'reads': [190598116,
   190598116,
   190598116,
   190598125,
   190598125,
   190598240,
   190598424]},
 {'chr': '9',
  'start': 129077,
  'end': 130282,
  'num_reads': 3,
  'reads': [129577, 129768, 129782]},
 {'chr': '17',
  'start': 22020194,
  'end': 22021194,
  'num_reads': 16,
  'reads': [22020694,
   22020694,
   22020694,
   22020694,
   22020694,
   22020694,
   22020694,
   22020694,
   22020694,
   22020694,
   22020694,
   22020694,
   22020694,
   22020694,
   22020694,
   22020694]},
 {'chr': '10',
  'start': 42398123,
  'end': 42399249,
  'num_reads': 2,
  'reads': [42398623, 42398749]},
 {'chr': '11',
  'start': 49882868,
  'end': 49884072,
  'num_reads': 6,
  'reads': [49883368, 49883380, 49883399, 49883419, 49883495, 49883572]}]

In [41]:
# 示例：查看前5个候选区域
for i, region in enumerate(candidate_regions[:5]):
    print(f"区域 {i+1}: {region['chr']}:{region['start']}-{region['end']} (读段数={region['num_reads']})")


区域 1: 4:190597616-190598924 (读段数=7)
区域 2: 9:129077-130282 (读段数=3)
区域 3: 17:22020194-22021194 (读段数=16)
区域 4: 10:42398123-42399249 (读段数=2)
区域 5: 11:49882868-49884072 (读段数=6)


In [42]:
# 过滤掉读段数不足的区域
valid_regions = [r for r in candidate_regions if r['num_reads'] >= 2]
print("有效候选区域数量:", len(valid_regions))


有效候选区域数量: 5


In [43]:
# 为每个候选区域提取相关读段
for region in valid_regions:
    chr_name = region['chr']
    start = region['start']
    end = region['end']
    
    # 提取该区域的读段
    region_reads = df_filtered[
        (df_filtered['RNAME'] == chr_name) &
        (df_filtered['POS'].between(start, end))
    ]
    
    # 关联MT端的位置
    mt_positions = region_reads[region_reads['RNEXT'] == 'MT']['PNEXT'].astype(int).tolist()
    region['mt_positions'] = mt_positions

# 示例输出
print("示例候选区域:")
print({
    'chr': valid_regions[0]['chr'],
    'start': valid_regions[0]['start'],
    'end': valid_regions[0]['end'],
    'num_reads': valid_regions[0]['num_reads'],
    'mt_positions': valid_regions[0]['mt_positions']
})


示例候选区域:
{'chr': '4', 'start': 190597616, 'end': 190598924, 'num_reads': 7, 'mt_positions': [12361, 13169, 12361, 13173, 12361, 12361, 12376]}


In [44]:
# 转换为DataFrame并保存
df_candidates = pd.DataFrame(valid_regions)
df_candidates = df_candidates[['chr', 'start', 'end', 'num_reads', 'mt_positions']]
df_candidates


,chr,start,end,num_reads,mt_positions
0,4,190597616,190598924,7,"[12361, 13169, 12361, 13173, 12361, 12361, 12376]"
1,9,129077,130282,3,"[6275, 6347, 6228]"
2,17,22020194,22021194,16,"[16302, 16337, 16235, 16200, 31, 16229, 16394,..."
3,10,42398123,42399249,2,"[2249, 4927]"
4,11,49882868,49884072,6,"[16463, 16429, 16307, 16349, 16369, 16089]"


In [53]:
# 1) 把原来的 num_reads 重命名成 Cluster_No
df_candidates = df_candidates.rename(
    columns={'num_reads': 'Cluster_No'}
)

In [ ]:
# 2) 补齐其它几列  
df_candidates['IndividualID'] = "Y21100000492606.deduped"  
df_candidates['disFile']      = input_disc  
df_candidates['splitFile']    = input_disc.replace('disc', 'split')  
df_candidates['wgsBAM']       = "/home/luolintao/S20_NUMTs-detection-1.0/data/Y21100000492606.deduped.bam"  

In [60]:
# 2.1) 计算 subCluster_No：MT 端落点数目  
df_candidates['subCluster_No'] = df_candidates['mt_positions'].apply(len)  

In [61]:
# 2.2) 计算 Cluster_ID  
#    注意：原脚本里用的是 POS 最小/最大，再加上 MT 端 PNEXT 的最小/最大  
#    因为你的 start = min(POS)-500, end = max(POS)+500，
#    这里我们反推回去：
df_candidates['Cluster_ID'] = df_candidates.apply(
    lambda row: (
        f"{row['chr']}_"
        f"{row['start']+500}_"
        f"{row['end']-500}_MTboth_"
        f"{min(row['mt_positions'])}_"
        f"{max(row['mt_positions'])}"
    ),
    axis=1
)

In [62]:
# 2.3) 现在把列都排齐，为 summary 做准备  
summary_keys = ['IndividualID','Cluster_ID','Cluster_No','subCluster_No']  
df_summary = (
    df_candidates
    .groupby(summary_keys)
    .size()
    .to_frame('size')
    .reset_index()
)

In [56]:
# 3) 选列并排序
cols = [
    'IndividualID',
    'Cluster_No',
    'disFile',
    'splitFile',
    'wgsBAM',
    'chr',
    'start',
    'end'
]
df_out = df_candidates[cols]

In [57]:
# 4) 写文件：与第一个脚本完全一致的格式
output_file = input_disc + '.breakpointINPUT.tsv'
df_out.to_csv(
    output_file,
    sep='\t',
    header=False,   # 不输出列名
    index=False     # 不输出行索引
)

In [58]:
# .cluster.tsv —— 带列名、不带索引
df_candidates.to_csv(
    input_disc + '.cluster.tsv',
    sep='\t',
    header=True,
    index=False
)

In [63]:
# 4) .breakpointINPUT.tsv （和之前一样）
df_candidates[[
    'IndividualID','Cluster_No','disFile','splitFile','wgsBAM','chr','start','end'
]].to_csv(
    input_disc + '.breakpointINPUT.tsv',
    sep='\t', header=False, index=False
)

# 5) .cluster.tsv —— 带列名、不带索引
df_candidates.to_csv(
    input_disc + '.cluster.tsv',
    sep='\t', header=True, index=False
)

# 6) .cluster.summary.tsv —— 不带列名、带索引
df_summary.to_csv(
    input_disc + '.cluster.summary.tsv',
    sep='\t', header=False, index=True
)